In [2]:
import dfs_merge

/opt/conda/lib/python3.12/site-packages/cma/s.py:13: UserWarning: Could not import matplotlib.pyplot, therefore ``cma.plot()`` etc. is not available
  _warnings.warn('Could not import matplotlib.pyplot, therefore'


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, GenerationConfig
import torch
import torch.nn as nn
import numpy as np
from cma.evolution_strategy import CMAEvolutionStrategy

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.12/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
model_name = 'Vikhrmodels/it-5.4-fp16-orpo-v2'
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cpu",
    attn_implementation="sdpa",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.54it/s]


In [5]:
model2 = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cpu",
    attn_implementation="sdpa",
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.46it/s]


In [6]:
M = 32
r = 1
n = 2
cma_es_I = CMAEvolutionStrategy([1] + [0] * (M * r * n - 2) + [1], 0.1)
cma_es_W = CMAEvolutionStrategy([0.5] * ((M * r * n + 2) * (M * r * n + 1) // 2), 0.5)

(8_w,16)-aCMA-ES (mu_w=4.8,w_1=32%) in dimension 64 (seed=326488, Thu Aug 22 00:40:18 2024)
(13_w,27)-aCMA-ES (mu_w=7.8,w_1=22%) in dimension 2145 (seed=335576, Thu Aug 22 00:40:18 2024)


In [7]:
POPULATION_SIZE = 2

In [8]:
Is = cma_es_I.ask(number=POPULATION_SIZE)
Ws = cma_es_W.ask(number=POPULATION_SIZE)

In [9]:
Wm = dfs_merge.W_to_matrix(Ws[0], M * n * r + 2)

In [10]:
(Is[0] > 0).astype(np.int8)

array([1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1],
      dtype=int8)

In [12]:
merged = dfs_merge.construct_merged_model([model, model2], (Is[0] > 0).astype(np.int8), Wm, 1, 32)

In [13]:
tokenizer("коты любят кашу")

{'input_ids': [1, 1619, 5555, 43812, 2790, 16672], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [14]:
torch.cuda.device_count()

4

In [15]:
inp = torch.tensor([[1, 1619, 5555, 43812, 2790, 16672]])

In [16]:
with torch.no_grad():
    emb = merged["embed"](inp)
    rot = merged["rotary"](emb, torch.tensor([[i for i in range(len(emb))]]))
    out = emb
    idx = 0
    for layer in merged["layers"]:
        if idx % 2 == 0:
            out = layer(emb)
        else:
            out = layer(hidden_states=emb, position_embeddings=rot)
        idx += 1
    out = merged["lm_head"](out)

In [17]:
out.shape

torch.Size([1, 6, 79100])